1. Put all paths, variables that are subjective to changes based on user needs to the beginning of your code or create a separate config.py file for them. People don't want to spend time to go deep in your code and find those paths one by one.

In [16]:
import geopandas as gpd
import numpy as np
import rasterio 
import rasterio.mask
from shapely.geometry import mapping


#from config import *

CDL_FMT = 'data/CDL/fips_{fips}/{year}/CDL_{year}.tif'
FIELD_BOUNDARY_FMT = 'data/field_boundary/hb-{fips}/hb-{fips}.shp'

fips = '17019'
years = range(2017,2022)


2. Add function definition for each of the function/class you wrote.
3. Add proper comments to: (1) describe a complicated logic, (2) summarize a subsection of code.
4. Optimize your code. (1) avoid loops, and try to use built-in functions to maximize performances. (2) Eliminate duplications of codes. (3) Delete unnecessary logging, printing in your code.
5. Naming of variables should be clear, and should not cause confusion.

In [17]:
def extract_field_cdl(fips,years):
    """Extract crop type for all fields in target county using CDL.
        Parameters:
            fips (string): county fips code.
            years (int years): crop years needed.
        return:
            crop_type (dictionary): year -> unique_fid -> crop code
    """
    
    field_df = gpd.read_file(FIELD_BOUNDARY_FMT.format(fips=fips))
    
    crop_type = {}
    
    for year in years:
        crop_type[year] ={}
        cdl_ds =  rasterio.open(CDL_FMT.format(fips=fips,year=year))
        #Make sure your vector file crs is the same as your raster file cr
        if(field_df.crs != cdl_ds.crs):
            field_df = field_df.to_crs(cdl_ds.crs)
        
        #Iterate through fields to extract crop type.
        for i,row in field_df.iterrows():
            masked_cdl,_ = rasterio.mask.mask(cdl_ds,[mapping(row.geometry)], crop=True, nodata=0) #0 is not used in CDL so we can use it as nodata values for pixels outside field boundary
            masked_cdl = masked_cdl[0]
            masked_cdl = masked_cdl[masked_cdl!=0]
            crop_type[year][row['unique_fid']] = np.argmax(np.bincount(masked_cdl)) #Take mode
            
    return crop_type

In [18]:
ct = extract_field_cdl(fips,years)